# Malaria Diseased Cell Classification

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from random import shuffle
from tqdm import tqdm

from keras.models import Sequential
from keras.models import model
from keras.layers import *
from keras.optimizers import *

Using TensorFlow backend.


#### Define path to database

In [2]:
cwd = os.getcwd()  #Get Path of Current Working Directory

infected_train_data = cwd + '/cell_images/Train/infected'
uninfected_train_data = cwd + '/cell_images/Train/uninfected'
infected_test_data = cwd + '/cell_images/Test/infected'
uninfected_test_data = cwd + '/cell_images/Test/uninfected'

#### Define Hyper-parameters

In [3]:
num_epochs = 50
size_batch = 50
alpha = 0.0001

In [4]:
def one_hot_label(img, label):
    if label == 0:
        ohl = np.array([1,0])
    elif label == 1:
        ohl = np.array([0,1])
    return ohl

def load_train_data():
    train_images = []
    
    label = 0  #For Infected Cells
#     print("Loading Training data for infected Cells")
    for i in tqdm(os.listdir(infected_train_data)):
        path = os.path.join(infected_train_data, i)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, (64,64))
        except:
            print("Error Loading this image: ", i) 
        train_images.append([np.array(img), one_hot_label(i, label)])
    
    label = 1 #For Uninfected
#     print("Loading Training data for uninfected Cells")
    for i in tqdm(os.listdir(uninfected_train_data)):
        path = os.path.join(uninfected_train_data, i)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, (64,64))
        except:
            print("Error Loading this image: ", i) 
        train_images.append([np.array(img), one_hot_label(i, label)])
    
    shuffle(train_images) #Shuffle data so every batch gets both, positives and negatives.
    shuffle(train_images) #Just to be sure Lol!
    return train_images

def load_test_data():
    test_images = []
    
    label = 0
#     print("Loading Test data for infected Cells")
    for i in tqdm(os.listdir(infected_test_data)):
        path = os.path.join(infected_test_data, i)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, (64,64))
        except:
            print("Error Loading this image: ", i)            
        test_images.append([np.array(img), one_hot_label(i, label)])
    
    label = 1
#     print("Loading Test data for uninfected Cells")
    for i in tqdm(os.listdir(uninfected_test_data)):
        path = os.path.join(uninfected_test_data, i)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, (64,64))
        except:
            print("Error Loading this image: ", i)
        test_images.append([np.array(img), one_hot_label(i, label)])
    return test_images   

In [5]:
training_images = load_train_data()
testing_images = load_test_data()

x_train = []
y_train = []

for k in training_images:
    x_train.append(k[0])
    y_train.append(k[1])

x_train = np.array(x_train)
y_train = np.array(y_train)

100%|██████████| 1000/1000 [00:00<00:00, 1947.68it/s]


### Define Model

In [6]:
model = Sequential()

model.add(InputLayer(input_shape=[64,64,3]))
model.add(Conv2D(filters = 32, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters = 50, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters = 80, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))
optimizer = Adam(lr = alpha)


model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:


model.fit(x=x_train, y=y_train, epochs = num_epochs, batch_size = size_batch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
25558/25558 [==============================] - 29s 1ms/step - loss: 1.2490 - acc: 0.5832
Epoch 2/50
25558/25558 [==============================] - 9s 333us/step - loss: 0.5598 - acc: 0.7117
Epoch 3/50
25558/25558 [==============================] - 8s 331us/step - loss: 0.4227 - acc: 0.8045
Epoch 4/50
25558/25558 [==============================] - 8s 328us/step - loss: 0.3071 - acc: 0.8676
Epoch 5/50
25558/25558 [==============================] - 8s 329us/step - loss: 0.2222 - acc: 0.9143
Epoch 6/50
25558/25558 [==============================] - 8s 330us/step - loss: 0.1913 - acc: 0.9289
Epoch 7/50
25558/25558 [==============================] - 9s 334us/step - loss: 0.1696 - acc: 0.9391
Epoch 8/50
25558/25558 [==============================] - 8s 331us/step - loss: 0.1558 - acc: 0.9441
Epoch 9/50
25558/25558 [==============================] - 9s 334us/step - loss: 0.1475 - acc: 0.9500
Epoch 10/50
25558/25558 [===================

In [11]:
scores = model.evaluate(x_train, y_train, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.77%


#### Save Model

In [12]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
print("Saved Model as JSON file")

Saved Model as JSON file


### Testing

In [ ]:
fig = plt.fig(figsize = (14,14))
